In [1]:
import time
import os
import openai
import sys
import os
import json
import random
import gym
import numpy as np
from typing import List, Dict, Any

import random
import torch
from torch.utils.data import DataLoader
from transformers import (
    AutoTokenizer, AutoModelForCausalLM, AdamW
)
from accelerate import Accelerator
import random
import torch
from torch.utils.data import DataLoader
from transformers import (
    AutoTokenizer, AutoModelForCausalLM, AdamW
)
from accelerate import Accelerator

from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig
from trl import GRPOConfig, GRPOTrainer
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from sklearn.model_selection import train_test_split

In [2]:
#client = OpenAI()

In [3]:
!nvidia-smi

Wed Feb  5 17:50:57 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.171.04             Driver Version: 535.171.04   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A6000               On  | 00000000:2D:00.0 Off |                  Off |
| 30%   51C    P8              17W / 300W |     13MiB / 49140MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
class LMFunction(object):
    def __init__(self, engine='gpt-4', max_tokens=512):
        self.engine = engine
        self.max_tokens = max_tokens
        self.client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

    def _call_api(self, prompt, engine, max_tokens, max_retries=10, retry_wait=2):
        for i in range(max_retries):
            try:
                response = self.client.chat.completions.create(
                    model=engine,
                    messages=[
                        {"role": "system", "content": "You are a helpful assistant."},
                        {"role": "user", "content": prompt}
                    ],
                    max_tokens=max_tokens,
                    temperature=1.0
                )
                return response
            except openai.APIError as e:  # New error handling
                time.sleep(retry_wait)
        return {'choices': [{'message': {'content': ''}}]}

    def _parse_message(self, msg):
        try:
            content = msg.choices[0].message.content
        except (IndexError, AttributeError):
            content = ''
        return content

    def f(self, prompt, x):
        msg = self._call_api(
            prompt=prompt+x,
            engine=self.engine,
            max_tokens=self.max_tokens
        )
        evaluation = self._parse_message(msg)
        return evaluation


class Checker(object):
    """A modified version of the Draft, Sketch, Prove proof-checking client.
    (https://github.com/albertqjiang/draft_sketch_prove/blob/main/autoformalization/checker.py)

    This checker supports Isabelle2022 via the new version of PISA
    (https://albertqjiang.github.io/Portal-to-ISAbelle/).

    It supports checking a miniF2F-style proof via `check`.

    Finally, it replaces `sledgehammer` with a call to `normalhammer`.
    """
    def __init__(self, working_dir, isa_path, theory_file, port=9000):
        sys.path.append(os.environ['PISA_PATH'])
        try:
            from pisa_client import initialise_env
            self.initialise_env = initialise_env
        except:
            print("Set $PISA_PATH to /yourpath/to/Portal-to-ISAbelle/src/main/python")

        self.working_dir = working_dir
        self.isa_path = isa_path
        self.theory_file = theory_file
        self.port = port

    def _initialize(self):
        env = self.initialise_env(
            self.port,
            isa_path=self.isa_path,
            theory_file_path=self.theory_file,
            working_directory=self.working_dir
        )
        return env

    def _exit(self, env):
        try:
            env.post('exit')
        except:
            print("env.post('exit') timed out")
            pass
        os.system("ps aux | grep Isabelle | awk '{print $2}' | xargs kill -9 > /dev/null 2>&1")
        os.system("ps aux | grep poly | awk '{print $2}' | xargs kill -9 > /dev/null 2>&1")

    def _parse_output(self, obs):
        """Parse the sledgehammer output, otherwise return an empty string"""
        if '<hammer>' in obs:
            output = obs.split('<hammer>')[0]
        else:
            output = ''
        return output

    def _run_step(self, step, i, tls_name, env):
        obs, reward, done, metadata = env.step_to_top_level_state(
            action=step,
            tls_name=tls_name,
            new_name='default_%d' % i
        )
        error = None
        if 'error:' in obs or 'Step error' in obs or 'Unknown error' in obs:
            error = obs
        return obs, reward, done, metadata, error

    def _run_sledgehammer(self, step, i, tls_name, env):
        # First try heuristics
        for heuristic in ['by auto', 'by simp', 'by blast', 'by fastforce', 'by force', 'by eval', 'by presburger', 'by sos', 'by arith', 'by linarith', 'by (auto simp: field_simps)']:
            step_ = step.replace('normalhammer', heuristic)
            obs, reward, done, metadata, error = self._run_step(step_, i, tls_name, env)
            if error is None:
                obs = '%s <hammer> %s' % (heuristic, obs)
                return obs, reward, done, metadata, error
        # Try sledgehammer
        out = self._run_step(step, i, tls_name, env)
        return out

    def check(self, statement_and_proof):
        # Initialize environment
        env = self._initialize()
        env.initialise()

        # Wrap and parse theorem
        theory = Checker.wrap_theorem(statement_and_proof)
        steps = Checker.get_parsed(env, theory)

        result = self._check(env, steps)
        return result

    def _check(self, env, steps):
        done = False
        reason = ''
        success = False
        step_results = []
        tls_name = 'default'
        for i, step in enumerate(steps):
            try:
                time0 = time.time()
                if 'normalhammer' in step:
                    obs, reward, done, metadata, error = self._run_sledgehammer(step, i, tls_name, env)
                else:
                    obs, reward, done, metadata, error = self._run_step(step, i, tls_name, env)
                step_time = time.time() - time0
                step_results.append(dict(index=i, step=step, output=self._parse_output(obs), step_time=step_time))
                if error is not None:
                    reason = error
                    success = False
                    done = False
                    break
            except:
                # Timeout - end the proof attempt
                success = False
                done = False
                reason = 'timeout (%d)' % len(step_results)
                step_results.append(dict(index=i, step=step, output=''))
                break

            # Change when successful
            tls_name = 'default_%d' % i

        if done and reward == 1.0:
            success = True

        result = {
            'success': success,
            'reason': reason,
            'num_steps': len(steps),
            'last_step': len(step_results),
            'step_results': step_results,
            'theorem_and_proof': self.reconstruct(step_results) if success else ''
        }
        # Exit environment
        self._exit(env)
        return result
    
    @staticmethod
    def reconstruct(step_results):
        steps = []
        for step_result in step_results[1:]:
            if step_result['output'] != '':
                steps.append(step_result['output'].strip())
            else:
                steps.append(step_result['step'].strip())
        theorem_and_proof = '\n'.join(steps)
        return theorem_and_proof

    @staticmethod
    def wrap_theorem(theorem):
        return 'theory Interactive imports HOL.HOL Complex_Main "HOL-Library.Code_Target_Numeral" "HOL-Library.Sum_of_Squares" "Symmetric_Polynomials.Vieta" "HOL-Computational_Algebra.Computational_Algebra" "HOL-Number_Theory.Number_Theory" \n begin\n%s' % theorem

    @staticmethod
    def get_parsed(env, theory, tls_name='default'):
        # HACK: the parsing doesn't work well with `normalhammer`, so we replace
        # all hammer calls with sorry, then replace sorry to normalhammer after parsing.
        theory = theory.replace('sledgehammer', 'sorry')
        theory = theory.replace('normalhammer', 'sorry')

        steps = env.post(f"<parse text> ${theory}")
        steps = steps.split('<SEP>')
        steps = [s for s in steps if s.strip() != '']
        # remove weird '$' step and whitespace steps
        steps = [s for s in steps if s != '$' and s.strip() != '']
        steps = [s.replace('sorry', 'normalhammer') for s in steps]
        return steps


In [5]:
import sys
import os
sys.path.append('../')
os.environ['PISA_PATH'] = '/home/siai/Portal-to-ISAbelle/src/main/python'

checker = Checker(
    working_dir='/home/siai/Isabelle2022/src/HOL/Examples',
    isa_path='/home/siai/Isabelle2022',
    theory_file='/home/siai/Isabelle2022/src/HOL/Examples/Interactive.thy',
    port=9000
)

In [6]:
def load_theorem_data(json_path: str) -> List[Dict[str, str]]:
    """
    Loads a JSON file containing an array of objects with:
      { "statement": "...", "state": "...", "step": "..." }
    Returns a list of dicts with these fields.
    """
    with open(json_path, 'r') as f:
        data = json.load(f)
    return data


def load_train_val_data(json_path: str, test_size=0.1, random_seed=42):
    with open(json_path, 'r') as f:
        data = json.load(f)
    train_data, val_data = train_test_split(data, test_size=test_size, random_state=random_seed)
    return train_data, val_data    

In [7]:
def evaluate_validation_loss(model, val_loader):
    model.eval()
    total_loss = 0.0
    total_count = 0
    with torch.no_grad():
        for batch in val_loader:
            for k, v in batch.items():
                batch[k] = v.cuda()
            outputs = model(**batch)
            loss = outputs.loss
            batch_size = batch["input_ids"].size(0)
            total_loss += loss.item() * batch_size
            total_count += batch_size
    model.train()
    return total_loss / total_count

In [8]:
def build_trl_format(dataset: List[Dict[str,str]]) -> List[Dict[str,Any]]:
    """
    Convert each record { "statement", "state", "step" } into TRL's conversation format:
      "prompt": [
        {"role": "system", "content": "Prove the following theorem."},
        {"role": "user", "content": statement + "\n\n" + state}
      ],
      "answer": step

    This single-turn approach: the user "says" the lemma & subgoal, the model must produce the final "step".
    """
    trl_data = []
    for rec in dataset:
        stm = rec.get("statement","")
        stt = rec.get("state","")
        sp  = rec.get("step","")
        prompt_content = stm + "\n\n" + stt
        sample = {
            "prompt": [
                {"role": "system", "content": "Prove the following theorem."},
                {"role": "user",   "content": prompt_content}
            ],
            "answer": sp  
        }
        trl_data.append(sample)
    return trl_data

In [9]:
class OfflineImitationDataset(Dataset):
    """
    For supervised training: input = statement + "\n\n" + state
                            target = step
    """
    def __init__(self, raw_data: List[Dict[str,str]]):
        super().__init__()
        self.samples = []
        for rec in raw_data:
            inp = rec["statement"] + "\n\n" + rec["state"]
            tgt = rec["step"]
            self.samples.append((inp, tgt))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]

In [10]:
def offline_collate_fn(batch, tokenizer, max_length=512):
    inp_texts, tgt_texts = zip(*batch)
    enc = tokenizer(
        list(inp_texts),
        text_target=list(tgt_texts),
        padding='max_length',
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )
    return {
        "input_ids": enc["input_ids"],
        "attention_mask": enc["attention_mask"],
        "labels": enc["labels"]
    }


In [11]:
def run_offline_imitation_learning(
    json_path: str,
    model_name: str,
    output_dir: str,
    epochs: int = 1,
    lr: float = 1e-5,
    max_length: int = 512,
    batch_size: int = 2
):

    # Load data
    raw_data = load_theorem_data(json_path)
    dataset_obj = OfflineImitationDataset(raw_data)

    # Load model & tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    #tokenizer.pad_token = tokenizer.eos_token
    model = AutoModelForCausalLM.from_pretrained(model_name)
    
    tokenizer.add_special_tokens({"pad_token": "<PAD>"})
    model.resize_token_embeddings(len(tokenizer))
    tokenizer.pad_token = "<PAD>"
    
    model.config.use_cache = False   
    model.train()
    model.cuda()

    # Build DataLoader
    loader = DataLoader(
        dataset_obj,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=lambda b: offline_collate_fn(b, tokenizer, max_length)
    )

    optimizer = AdamW(model.parameters(), lr=lr)

    # Train loop
    step_idx = 0
    for epoch in range(epochs):
        for batch in loader:
            for k, v in batch.items():
                batch[k] = v.cuda()

            outputs = model(**batch)
            loss = outputs.loss
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            step_idx += 1
            print(f"Epoch {epoch} Step {step_idx}, loss={loss.item():.4f}")

    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    print(f"Offline training done, saved to {output_dir}")

In [12]:
def generate_proof_step(model, tokenizer, statement, state, max_length=32):
    # Build the input text for the model
    input_text = statement + "\n\n" + state
    # Encode
    enc_in = tokenizer.encode(input_text, return_tensors="pt").cuda()
    # Generate
    out_ids = model.generate(
        enc_in,
        max_new_tokens=max_length,
        do_sample=False,   # or False for  greedy
        top_p=0.9,        
        temperature=0.8,  # or 1.0
    )
    # Decode
    out_text = tokenizer.decode(out_ids[0], skip_special_tokens=True)
    return out_text

In [13]:
def checker_reward_func(prompts, completions, answer, checker, **kwargs) -> List[float]:
    """
    Single-step approach: 
      prompts[i][-1]['content'] = statement+state
      completions[i][0]['content'] = model's final step
    We unify them, pass to checker, parse partial or full success => returns float.
    """
    rewards = []
    for i in range(len(prompts)):
        theorem_text = prompts[i][-1]['content']
        step_text = completions[i][0]['content']
        combined = f"{theorem_text}\n\n{step_text}"
        result = checker.check(combined)
        reason = result["reason"]
        if result["success"]:
            # Full success
            rewards.append(2.0)
        elif reason == "partial":
            rewards.append(0.5)
        else:
            rewards.append(0.0)
    return rewards


In [14]:
def format_reward_func(completions, **kwargs) -> List[float]:
    """
    For demonstration, add a small reward if the output contains "by " or "sledgehammer " or "normalhammer" .
    """
    outs = []
    for c in completions:
        txt = c[0]["content"].lower()
        if "by " or "sledgehammer " or "normalhammer" in txt:
            outs.append(0.1)
        else:
            outs.append(0.0)
    return outs

In [15]:
def train_with_early_stopping(
    model,
    train_loader,
    val_loader,
    optimizer,
    max_global_steps=100000,
    eval_every=100,
    patience=3
):
    """
    Train the model, validating every 'eval_every' steps.
    Stop early if validation fails to improve for 'patience' intervals.

    :param model: your HF model
    :param train_loader: DataLoader for training
    :param val_loader: DataLoader for validation
    :param optimizer: optimizer
    :param max_global_steps: max steps we allow in total
    :param eval_every: how often (in steps) to run validation
    :param patience: how many times in a row we allow no improvement
    """

    best_val_loss = float("inf")
    no_improvement_count = 0
    global_step = 0
    model.train()

    for epoch in range(999999):  # effectively "infinite" until we break
        for step_idx, batch in enumerate(train_loader):
            global_step += 1

            for k, v in batch.items():
                batch[k] = v.cuda()
            outputs = model(**batch)
            loss = outputs.loss
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if global_step % 50 == 0:
                print(f"[Global Step {global_step}] training loss={loss.item():.4f}")

            if global_step % eval_every == 0:
                val_loss = evaluate_validation_loss(model, val_loader)  # your function
                print(f"[Global Step {global_step}] val_loss={val_loss:.4f}")

                if val_loss < best_val_loss:
                    print("Validation improved!")
                    best_val_loss = val_loss
                    no_improvement_count = 0
                    # optionally save checkpoint
                    model.save_pretrained("checkpoint_best")
                else:
                    no_improvement_count += 1
                    print(f"No improvement count={no_improvement_count}")

                # ---- Early stopping if patience exceeded ----
                if no_improvement_count >= patience:
                    print("Early stopping: no improvement in val_loss for too long.")
                    return  # or break out of loops

            # ---- End if we exceed max steps ----
            if global_step >= max_global_steps:
                print("Reached maximum global steps.")
                return

In [16]:
def main():
    JSON_DATASET_PATH = "deduplicated_dataset.json"
    MODEL_NAME = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"
    OFFLINE_SAVE_DIR = "offline_ckpt"
    OFFLINE_EPOCHS = 2
    BATCH_SIZE = 2
    LR = 1e-5
    MAX_LENGTH = 512
    EVAL_EVERY = 1000
    PATIENCE = 3

    # Initialize Accelerator
    accelerator = Accelerator()
    device = accelerator.device

    # Load dataset
    train_data, val_data = load_train_val_data(JSON_DATASET_PATH, test_size=0.01)
    train_dataset = OfflineImitationDataset(train_data)
    val_dataset = OfflineImitationDataset(val_data)

    # Load model & tokenizer
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    tokenizer.pad_token = tokenizer.eos_token
    #model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        torch_dtype=torch.bfloat16,
        device_map="auto"  # Enables multi-GPU training
    )
    # Move model to Accelerator (multi-GPU, FP16 support)
    model.config.use_cache = False
    model.train()

    # Dataloader
    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        collate_fn=lambda b: offline_collate_fn(b, tokenizer, MAX_LENGTH)
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        collate_fn=lambda b: offline_collate_fn(b, tokenizer, MAX_LENGTH)
    )

    optimizer = AdamW(model.parameters(), lr=LR)

    # Prepare everything for Accelerator
    model, optimizer, train_loader, val_loader = accelerator.prepare(
        model, optimizer, train_loader, val_loader
    )

    # Training Variables
    global_step = 0
    best_val_loss = float("inf")
    no_improvement_count = 0
    stop_early = False

    for epoch in range(OFFLINE_EPOCHS):
        if stop_early:
            break

        model.train()
        for step_idx, batch in enumerate(train_loader):
            global_step += 1

            # Move batch to the correct device
            with accelerator.accumulate(model):
                outputs = model(**batch)
                loss = outputs.loss

                # Backpropagation
                optimizer.zero_grad()
                accelerator.backward(loss)
                optimizer.step()

            # Print training loss every 10 steps
            if step_idx % 10 == 0:
                print(f"Epoch {epoch}, step {step_idx}, train loss={loss.item():.4f}")

            # Generate a sample output every 500 steps
            if step_idx % 500 == 0 and len(val_data) > 0:
                sample = random.choice(val_data)
                gen_step = generate_proof_step(model, tokenizer, sample["statement"], sample["state"])
                print("-"*50)
                print(f"Sample statement+state:\n {sample['statement']} {sample['state']}")
                print("\n\nGenerated step:\n", gen_step)
                print("\n\nReference step:\n", sample["step"])
                print("-"*50)

            # Validation after every EVAL_EVERY steps
            if global_step % EVAL_EVERY == 0:
                model.eval()
                val_loss = evaluate_validation_loss(model, val_loader)
                print(f"[Global Step {global_step}] Interim val_loss={val_loss:.4f}")

                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    no_improvement_count = 0
                    print("  (New best val_loss!)")
                    accelerator.unwrap_model(model).save_pretrained("checkpoint_best")
                else:
                    no_improvement_count += 1
                    print(f"  (No improvement. Count={no_improvement_count})")
                    if no_improvement_count >= PATIENCE:
                        print("Early stopping triggered (no val improvement).")
                        stop_early = True
                        break
                model.train()

        # End of epoch => Final validation
        if not stop_early:
            model.eval()
            val_loss = evaluate_validation_loss(model, val_loader)
            print(f"Epoch {epoch}, validation loss={val_loss:.4f}")

            if len(val_data) > 0:
                sample = random.choice(val_data)
                gen_step = generate_proof_step(model, tokenizer, sample["statement"], sample["state"])
                print("Sample statement+state:\n", sample["statement"], sample["state"])
                print("Generated step:\n", gen_step)
                print("Reference step:\n", sample["step"])
            model.train()

    # Save final model
    accelerator.unwrap_model(model).save_pretrained(OFFLINE_SAVE_DIR)
    tokenizer.save_pretrained(OFFLINE_SAVE_DIR)
    print(f"Offline training done. Model saved to: {OFFLINE_SAVE_DIR}")



In [17]:
if __name__ == "__main__":
    main()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Epoch 0, step 0, train loss=15.9926


/home/siai/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/siai/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may 

--------------------------------------------------
Sample statement+state:
 lemma rt_fresh_as_inc_invalidate [simp]:
  assumes "dip\<in>kD(rt)"
      and "\<forall>rip\<in>dom(dests). rip\<in>vD(rt) \<and> the (dests rip) = inc (sqn rt rip)"
    shows "rt \<approx>\<^bsub>dip\<^esub> invalidate rt dests" proof (chain)
picking this:
  dip \<in> vD rt


Generated step:
 lemma rt_fresh_as_inc_invalidate [simp]:
  assumes "dip\<in>kD(rt)"
      and "\<forall>rip\<in>dom(dests). rip\<in>vD(rt) \<and> the (dests rip) = inc (sqn rt rip)"
    shows "rt \<approx>\<^bsub>dip\<^esub> invalidate rt dests"

proof (chain)
picking this:
  dip \<in> vD rt
  the (dests dip) = inc (sqn rt dip)
  dip = dip
  the (dests dip) = inc (sq


Reference step:
 using vD_nsqn_sqn by presburger
--------------------------------------------------
Epoch 0, step 10, train loss=7.0627
Epoch 0, step 20, train loss=1.7336
Epoch 0, step 30, train loss=0.4751
Epoch 0, step 40, train loss=1.5488
Epoch 0, step 50, train loss=

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


...

Epoch 0, step 9500, train loss=0.5816
--------------------------------------------------
Sample statement+state:
 lemma Exit_path_Low_path:
  assumes "n -as\<rightarrow>* (_Exit_)" and "inner_node n"
  obtains a' as' where "as = as'@[a']" and "n -as'\<rightarrow>* (_Low_)"
  and "kind a' = (\<lambda>s. True)\<^sub>\<surd>" 


Generated step:
 lemma Exit_path_Low_path:
  assumes "n -as\<rightarrow>* (_Exit_)" and "inner_node n"
  obtains a' as' where "as = as'@[a']" and "n -as'\<rightarrow>* (_Low_)"
  and "kind a' = (\<lambda>s. True)\<^sub>\<surd>"




Reference step:
 by (metis Exit_edge_Entry_or_Low Low_source_Exit_edge assms(1) assms(2) edge_det inner_node_def list.distinct(1) path.cases path_Entry_target(1) path_split(1) path_split(2) path_split(3) rev_exhaust)
--------------------------------------------------
Epoch 0, step 9510, train loss=0.1502
Epoch 0, step 9520, train loss=0.1701
Epoch 0, step 9530, train loss=0.1277
Epoch 0, step 9540, train loss=0.1272
Epoch 0, step 9550, 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Epoch 0, step 10000, train loss=0.1519
--------------------------------------------------
Sample statement+state:
 instance atoms :: (atomic_boolean_algebra) atomic_boolean_algebra proof (state)
this:
  \<exists>y. x = at_map y \<and> x \<noteq> \<bottom>

goal (1 subgoal):
 1. \<And>x.
       x \<noteq> \<bottom> \<Longrightarrow>
       \<exists>y. atom y \<and> y \<le> x


Generated step:
 instance atoms :: (atomic_boolean_algebra) atomic_boolean_algebra

proof (state)
this:
  \<exists>y. x = at_map y \<and> x \<noteq> \<bottom>

goal (1 subgoal):
 1. \<And>x.
       x \<noteq> \<bottom> \<Longrightarrow>
       \<exists>y. atom y \<and> y \<le> x


Reference step:
 by (metis at_map_bot_pres atomicity)
--------------------------------------------------
Epoch 0, step 10010, train loss=0.2537
Epoch 0, step 10020, train loss=0.1449
Epoch 0, step 10030, train loss=0.2462
Epoch 0, step 10040, train loss=0.3909
Epoch 0, step 10050, train loss=0.2666
Epoch 0, step 10060, train loss=0.4367


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Epoch 0, step 10500, train loss=0.1071
--------------------------------------------------
Sample statement+state:
 lift_definition bot_ennreal :: ennreal is 0 


Generated step:
 lift_definition bot_ennreal :: ennreal is 0




Reference step:
 by blast
--------------------------------------------------
Epoch 0, step 10510, train loss=0.1787
Epoch 0, step 10520, train loss=0.3180
Epoch 0, step 10530, train loss=0.1610
Epoch 0, step 10540, train loss=0.2209
Epoch 0, step 10550, train loss=0.1362
Epoch 0, step 10560, train loss=0.4391
Epoch 0, step 10570, train loss=0.1698
Epoch 0, step 10580, train loss=0.3066
Epoch 0, step 10590, train loss=0.2466
Epoch 0, step 10600, train loss=0.1958
Epoch 0, step 10610, train loss=0.4219
Epoch 0, step 10620, train loss=0.5444
Epoch 0, step 10630, train loss=0.1927
Epoch 0, step 10640, train loss=0.1724
Epoch 0, step 10650, train loss=0.1379
Epoch 0, step 10660, train loss=0.2490
Epoch 0, step 10670, train loss=0.2595
Epoch 0, step 10680, train loss=0

In [18]:
def main_grpo():
    """
    Multi-GPU RL function using TRL’s GRPO with:
      - Chunked incremental training
      - Sample output printing every 100 steps
      - Standard early stopping based on reward
      - RL validation on a held-out set
    """
    JSON_DATASET_PATH = "MagnusData/deduplicated_dataset.json"
    OFFLINE_CKPT = "checkpoint_best"  
    RL_SAVE_DIR = "outputs/Qwen-Theorem-GRPO"

    # Training settings
    EVAL_EVERY = 1000   
    PATIENCE = 3       
    MAX_GLOBAL_STEPS = 10000
    CHUNK_SIZE = 100    

    # Initialize accelerator for multi-GPU support
    accelerator = Accelerator()
    device = accelerator.device

    # Setup Isabelle Checker
    checker = Checker(
        working_dir='/home/siai/Isabelle2022/src/HOL/Examples',
        isa_path='/home/siai/Isabelle2022',
        theory_file='/home/siai/Isabelle2022/src/HOL/Examples/Interactive.thy',
        port=9000
    )

    # Load dataset & split
    train_data, val_data = load_train_val_data(JSON_DATASET_PATH, test_size=0.1)
    print(f"Train size: {len(train_data)}, Val size: {len(val_data)}")

    train_trl_data = build_trl_format(train_data)
    val_trl_data   = build_trl_format(val_data)

    # Load tokenizer & model
    tokenizer = AutoTokenizer.from_pretrained(OFFLINE_CKPT)
    tokenizer.pad_token = tokenizer.eos_token

    model = AutoModelForCausalLM.from_pretrained(
        OFFLINE_CKPT,
        torch_dtype=torch.bfloat16,
        device_map="auto"  # Enables multi-GPU training
    )

    # LoRA configuration (optional)
    peft_config = LoraConfig(
        r=4,
        lora_alpha=16,
        target_modules=["q_proj","k_proj","v_proj","o_proj","up_proj","down_proj","gate_proj"],
        task_type="CAUSAL_LM",
        lora_dropout=0.05,
    )

    # Reward function combining format + checker
    def checker_reward(prompts, completions, answer, **kwargs):
        return checker_reward_func(prompts, completions, answer, checker=checker)

    # Training arguments
    training_args = GRPOConfig(
        output_dir=RL_SAVE_DIR,
        run_name="Qwen-GRPO-theorems",
        learning_rate=1e-6,
        adam_beta1=0.9,
        adam_beta2=0.99,
        weight_decay=0.1,
        warmup_ratio=0.1,
        lr_scheduler_type="cosine",
        logging_steps=10,
        bf16=True,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=2,
        num_generations=1,
        max_prompt_length=512,
        max_completion_length=512,
        num_train_epochs=1,
        save_steps=9999999,
        max_grad_norm=0.1,
        report_to="wandb",
        log_on_each_node=False,
    )

    # Initialize Trainer with Accelerate
    trainer = GRPOTrainer(
        model=model,
        processing_class=tokenizer,
        reward_funcs=[format_reward_func, checker_reward],
        args=training_args,
        train_dataset=train_trl_data,
    )

    # Prepare trainer for multi-GPU training
    trainer.model, trainer.optimizer = accelerator.prepare(
        trainer.model, trainer.optimizer
    )

    best_val_reward = -9999
    no_improvement_count = 0
    global_step = 0
    stop_early = False

    # RL Training Loop
    while not stop_early and global_step < MAX_GLOBAL_STEPS:
        steps_to_run = min(CHUNK_SIZE, MAX_GLOBAL_STEPS - global_step)
        trainer.args.max_steps = steps_to_run
        trainer.args.num_train_epochs = 1

        # ---- Train for 'steps_to_run' steps
        trainer.train()
        global_step += steps_to_run
        print(f"Finished a block of {steps_to_run} RL steps, global_step={global_step}")

        # ---- Generate a sample output every chunk
        if len(val_data) > 0:
            sample = random.choice(val_data)
            prompt = [
                {"role": "system", "content": "Prove the following theorem."},
                {"role": "user", "content": sample["statement"] + "\n\n" + sample["state"]}
            ]

            completions = trainer.generate(
                [prompt], max_new_tokens=50, do_sample=False, top_p=0.9, temperature=0.8
            )
            gen_text = completions[0][0]['content'] if completions else ""

            print("-"*50)
            print("Sample statement+state:\n", sample["statement"], sample["state"])
            print("RL-generated step:\n", gen_text)
            print("Reference step:\n", sample["step"])
            print("-"*50)

        # RL Validation
        if global_step % EVAL_EVERY == 0:
            val_reward = evaluate_rl(trainer, val_trl_data, num_samples=200)
            print(f"[RL Validation] global_step={global_step}, val_reward={val_reward:.4f}")

            # Early stopping logic
            if val_reward > best_val_reward:
                best_val_reward = val_reward
                no_improvement_count = 0
                print("  (New best RL reward!)")
                accelerator.unwrap_model(trainer.model).save_pretrained("checkpoint_best_rl")
            else:
                no_improvement_count += 1
                print(f"  (No improvement. Count={no_improvement_count})")
                if no_improvement_count >= PATIENCE:
                    print("Early stopping triggered (RL val reward not improving).")
                    stop_early = True

    print(f"RL training complete at global_step={global_step}. Best val_reward={best_val_reward:.4f}")
    accelerator.unwrap_model(trainer.model).save_pretrained(RL_SAVE_DIR)
    print(f"RL final model saved in: {RL_SAVE_DIR}")

if __name__ == "__main__":
    main_grpo()


FileNotFoundError: [Errno 2] No such file or directory: 'MagnusData/deduplicated_dataset.json'

In [19]:
if __name__ == "__main__":
    main_grpo()

Train size: 333031, Val size: 37004


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

wandb: Currently logged in as: balaji-vir1997 (balaji-vir1997-stevens-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


/home/balaji/.local/lib/python3.9/site-packages/torch/amp/autocast_mode.py:44: FutureWarning: `num_logits_to_keep` is deprecated and will be removed in version 4.50 for `Qwen2ForCausalLM.forward`. Use `logits_to_keep` instead.
  return func(*args, **kwargs)
/home/balaji/.local/lib/python3.9/site-packages/trl/trainer/grpo_trainer.py:502: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /pytorch/aten/src/ATen/native/ReduceOps.cpp:1831.)
  std_grouped_rewards = rewards.view(-1, self.num_generations).std(dim=1)


Step,Training Loss


/pytorch/aten/src/ATen/native/cuda/TensorCompare.cu:110: _assert_async_cuda_kernel: block: [0,0,0], thread: [0,0,0] Assertion `probability tensor contains either `inf`, `nan` or element < 0` failed.


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
!nvidia-smi